In [1]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import AdamW, get_scheduler
from datasets import load_metric

import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split

from tqdm.auto import tqdm

import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

%matplotlib inline
%config InlineBackend.figure_format='retina'
sns.set(style='whitegrid', palette='muted', font_scale=1.2)
rcParams['figure.figsize'] = 16, 10

/root/.pyenv/versions/3.12.2/lib/python3.12/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
tokenizer = AutoTokenizer.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

model = AutoModelForSeq2SeqLM.from_pretrained("csebuetnlp/mT5_multilingual_XLSum")

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
/root/.pyenv/versions/3.12.2/lib/python3.12/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
/root/.pyenv/versions

In [3]:
df = pd.read_csv('text_sum.csv')
df['summary'] = df['summary'].replace(r'\n', '', regex = True)
df.head()

,text,summary
0,На фоне истощения запасов вооружения в западны...,На фоне истощения запасов вооружения в западны...
1,"ДОХА, 21 ноя —ПРАЙМ. Катарская национальная не...",Катарская национальная нефтегазовая компания Q...
2,Россия обсудит с ОНН вопрос о возобновлении по...,Россия обсудит с ООН вопрос о возобновлении по...
3,"Евгений Томихин указал, что эта тема обсуждает...",Посол РФ выразил надежду на подвижки в вопросе...
4,"МОСКВА, 23 ноя — ПРАЙМ. Индонезии требуются ин...",Индонезии требуются инвестиции в размере 179 м...


In [4]:
# text_token_counts = df['text'].apply(lambda x : len(tokenizer.encode(x)))
# summary_token_counts = df['summary'].apply(lambda x : len(tokenizer.encode(x)))

# fig, (ax1, ax2) = plt.subplots(1, 2)
# sns.boxplot(text_token_counts, ax=ax1)
# ax1.set_title('full text token counts')
# sns.boxplot(summary_token_counts, ax=ax2)
# ax2.set_title('summary text token counts')

In [5]:
class SummaryDataset(Dataset):
    def __init__(
        self,
        data=df,
        tokenizer=tokenizer,
        text_max_token_len = 768,
        summary_max_token_len = 200
    ):
        self.tokenizer = tokenizer
        self.data = data
        self.text_max_token_len = text_max_token_len
        self.summary_max_token_len = summary_max_token_len
    
    def __len__(self):
        return len(self.data)

    def __getitem__(self, index: int):
        data_row = self.data.iloc[index]

        text = data_row['text']

        text_encoding = tokenizer(
            text,
            max_length=self.text_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )

        summary_encoding = tokenizer(
            data_row['summary'],
            max_length=self.summary_max_token_len,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            add_special_tokens=True,
            return_tensors='pt'
        )
        
        labels = summary_encoding['input_ids']
        labels[labels == tokenizer.pad_token_id] = -100
        
        return dict(
            input_ids=text_encoding['input_ids'].flatten(),
            attention_mask=text_encoding['attention_mask'].flatten(),
            labels=labels.flatten(),
            decoder_attention_mask=summary_encoding['attention_mask'].flatten()
        )

In [6]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

train_dataset = SummaryDataset(data=df_train)
test_dataset = SummaryDataset(data=df_test)

train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=2)
eval_dataloader = DataLoader(test_dataset, batch_size=2)

In [7]:
torch.cuda.empty_cache()

In [8]:
num_epochs = 2

num_training_steps = num_epochs * len(train_dataloader)

optimizer = AdamW(model.parameters())
lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=num_training_steps
)

progress_bar = tqdm(range(num_training_steps))

device = torch.device("cuda:1")
model = model.to(device)

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}
        
        outputs = model(**batch)
#         logits = outputs.logits
#         predictions = torch.argmax(logits, dim=-1)
#         print(predictions)
#         print(batch["labels"])
        
        loss = outputs.loss
        loss.backward()
        
        optimizer.step()
        lr_scheduler.step()
        
        optimizer.zero_grad()
        progress_bar.update()
    
    torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss,
            }, f'./t5-epoch-{epoch}.pth')

    print(f'epoch: {epoch + 1} -- loss: {loss}')

  0%|          | 0/21536 [00:00<?, ?it/s]

epoch: 1 -- loss: 2.6793248653411865
epoch: 2 -- loss: 3.175328254699707


In [16]:
WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))

<>:1: SyntaxWarning: invalid escape sequence '\s'
<>:1: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_847823/391913681.py:1: SyntaxWarning: invalid escape sequence '\s'
  WHITESPACE_HANDLER = lambda k: re.sub('\s+', ' ', re.sub('\n+', ' ', k.strip()))


In [18]:
def get_sum(text):
    input_ids = tokenizer(
       text,
        return_tensors="pt",
        padding="max_length",
        truncation=True,
        max_length=512,
    )["input_ids"].to(device)

    output_ids = model.generate(
        input_ids=input_ids, no_repeat_ngram_size=2, num_beams=4
    )[0]

    summary = tokenizer.decode(
        output_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )

    return summary

In [20]:
text = """Россия и Китай намерены продолжать углубление взаимовыгодного сотрудничества в различных форматах на фоне нестабильности мировой экономики и западных санкций. Об этом говорится в совместном заявлении обеих стран по итогам девятого китайско-российского финансового диалога в Пекине, которое было принято 18 декабря. «Стороны будут углублять сотрудничество в рамках многосторонних форматов, включая G20, БРИКС и АТЭС, с целью ограничения рисков, связанных с геополитической и геоэкономической фрагментацией, последовательно продвигать экономическую глобализацию и поддерживать стабильность и бесперебойность глобальных промышленных цепочек и цепочек поставок», — цитирует данное заявление «РИА Новости». Отмечается, что развитие мировой экономики пока что остается неопределенным в связи с продолжительной и высокой инфляцией, последствиями пандемии коронавируса, геополитической напряженностью и политически мотивированными ограничениями. В связи с этим стороны договорились придерживаться в рамках сотрудничества принципов взаимной выгоды, а также подтвердили приверженность усилению экономических связей с помощью координации макроэкономической политики. Кроме того, РФ и КНР выразили поддержку расширения БРИКС, которое названо отправной точкой для наращивания усилий по финансово-экономическому сотрудничеству стран объединения. Ранее в этот день вице-премьер РФ Дмитрий Чернышенко заявил после заседания российско-китайской межправкомиссии по подготовке регулярных встреч глав правительств, что Россия и Китай планируют к 2030 году увеличить объем товарооборота до \$300 млрд. По его словам, за 11 месяцев 2023 года торговый оборот обеих стран уже превысил \$201 млрд. До этого, 15 декабря, полпред президента РФ в Дальневосточном федеральном округе (ДФО) Юрий Трутнев сообщил, что за 10 лет товарооборот между дальневосточными регионами России и северо-восточными провинциями Китая увеличился вдвое. По его данным, в 2022 году данный показатель составил \$22 млрд. Ожидается, что в 2023-м он достигнет \$27 млрд. В тот же день вице-премьер РФ Александр Новак сообщил, что Россия и Китай договорились о сотрудничестве в области технологий по производству водорода. В свою очередь, премьер Госсовета КНР Дин Сюэсян после встречи с Новаком также отметил, что энергетическое сотрудничество двух стран в текущем году активно развивалось и достигло новых результатов даже в условиях сложной и напряженной международной обстановки. 7 декабря президент России Владимир Путин заявил на пленарном заседании форума «Россия зовет!», что Москва готова к всестороннему сотрудничеству с Пекином. Он подчеркнул, что РФ и КНР думают о перспективах и будущем мироустройстве, а также отходят от модели «купил-продал» в двусторонних отношениях"""

summary = get_sum(text)

print(summary)

<>:1: SyntaxWarning: invalid escape sequence '\$'
<>:1: SyntaxWarning: invalid escape sequence '\$'
/tmp/ipykernel_847823/4206185205.py:1: SyntaxWarning: invalid escape sequence '\$'
  text = """Россия и Китай намерены продолжать углубление взаимовыгодного сотрудничества в различных форматах на фоне нестабильности мировой экономики и западных санкций. Об этом говорится в совместном заявлении обеих стран по итогам девятого китайско-российского финансового диалога в Пекине, которое было принято 18 декабря. «Стороны будут углублять сотрудничество в рамках многосторонних форматов, включая G20, БРИКС и АТЭС, с целью ограничения рисков, связанных с геополитической и геоэкономической фрагментацией, последовательно продвигать экономическую глобализацию и поддерживать стабильность и бесперебойность глобальных промышленных цепочек и цепочек поставок», — цитирует данное заявление «РИА Новости». Отмечается, что развитие мировой экономики пока что остается неопределенным в связи с продолжительной и

Один из ведущих китайских автопроизводителей Hyundai и Kia намерен расширить производство в Китае.
